In [1]:
import pandas as pd
import numpy as np
import random
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from rdkit import Chem
import sys
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
from minGPT.metrics import *
from minGPT.dataset import *
from minGPT.pipeline import *

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/home/cbe/anaconda3/envs/mingpt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/cbe/anaconda3/envs/mingpt/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped

In [2]:
import pandas as pd
data_final= pd.read_csv('minGPT/real_Tg_aug_data.csv',sep="\t")

In [3]:
data_final

,mol_smiles,Tg
0,C(*)*.C(C(OC(C)=O)*)*.70,0
1,C(*)*.*CC(*)OC(C)=O.70,0
2,C(*)*.*C(OC(C)=O)C*.70,0
3,C(*)*.C(OC(C*)*)(=O)C.70,0
4,C(*)*.O(C(=O)C)C(*)C*.70,0
...,...,...
112057,*C*.*CC(CCCCCCCCCBr)*.74,0
112058,*C*.C(C(*)CCCCCCCCCBr)*.74,0
112059,*C*.C(CCCCCCCCC(C*)*)Br.74,0
112060,*C*.C(Br)CCCCCCCCC(*)C*.74,0


In [4]:
from mingpt.model import GPT

# Data preprocessing
pipeline = minGPT()
data_config = pipeline.get_default_data_config()
data_config.file_path = "minGPT/real_Tg_aug_data.csv"
data_config.block_size = 192

print(data_config)
train_dataset, test_dataset = pipeline.data_preprocessing(data_config)

input_col: mol_smiles
length: 5
block_size: 192
train_test_split: (0.8, 0.2)
task: conditional
file_path: minGPT/real_Tg_aug_data.csv



In [5]:
# Model
model_config = pipeline.get_default_model_config()
#model_config.model_type = 'gpt-nano'
model_config.model_type = 'gpt-mini'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
pipeline.load_model(model_config)
print(model_config)

number of parameters: 2.82M
model_type: gpt-mini
n_layer: 6
n_head: 6
n_embd: 192
vocab_size: 592
block_size: 192
embd_pdrop: 0.1
resid_pdrop: 0.1
attn_pdrop: 0.1



In [6]:
# Train
train_config = pipeline.get_default_train_config()
print(train_config.device)
train_config.max_iters = 10000
train_config.ckpt_path = "./minGPT/ckpts_copolymer_real_Tg_aug/"

## Define call back function
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}, val loss {trainer.loss_val.item():.5f}")

train_config.call_back = batch_end_callback
print(train_config)
# loss = pipeline.train(train_config)

auto
device: auto
num_workers: 0
max_iters: 10000
batch_size: 64
learning_rate: 0.0005
betas: (0.9, 0.95)
weight_decay: 0.1
grad_norm_clip: 1.0
model: None
call_back: <function batch_end_callback at 0x7382f401fca0>
pretrain: None
ckpt_path: ./minGPT/ckpts_copolymer_real_Tg_aug/



In [7]:
loss = pipeline.train(train_config)

running on device cuda
iter_dt 0.00ms; iter 0: train loss 6.39100, val loss 6.39105
iter_dt 45.00ms; iter 100: train loss 0.35617, val loss 0.35721
iter_dt 46.76ms; iter 200: train loss 0.33752, val loss 0.33368
iter_dt 47.01ms; iter 300: train loss 0.26298, val loss 0.26234
iter_dt 43.07ms; iter 400: train loss 0.29417, val loss 0.29113
iter_dt 42.02ms; iter 500: train loss 0.25741, val loss 0.25984
iter_dt 41.96ms; iter 600: train loss 0.23936, val loss 0.23582
iter_dt 41.93ms; iter 700: train loss 0.26236, val loss 0.26477
iter_dt 42.09ms; iter 800: train loss 0.18991, val loss 0.18893
iter_dt 41.70ms; iter 900: train loss 0.25756, val loss 0.25566
iter_dt 45.74ms; iter 1000: train loss 0.20350, val loss 0.20657
iter_dt 45.42ms; iter 1100: train loss 0.20795, val loss 0.20886
iter_dt 45.08ms; iter 1200: train loss 0.19740, val loss 0.19771
iter_dt 46.22ms; iter 1300: train loss 0.20148, val loss 0.20660
iter_dt 46.05ms; iter 1400: train loss 0.18397, val loss 0.18256
iter_dt 42.88ms

In [8]:
# Generate
generate_config = pipeline.get_default_generate_config()
generate_config.ckpt_path = "./minGPT/ckpts_copolymer_real_Tg_aug/10000.pt"
assert generate_config.task == data_config.task
print(generate_config)

results = pipeline.generate(generate_config)

ckpts_path: None
num_samples: 5000
temperature: 1.0
task: conditional
ckpt_path: ./minGPT/ckpts_copolymer_real_Tg_aug/10000.pt



In [9]:
df_generate = pd.DataFrame(columns=['smiles1', 'smiles2', 'ratio'])
for item in results:

    if item.count('.') == 2:
       
        parts = item.split('.')
        
        new_row = pd.DataFrame({'smiles1': [parts[0]], 'smiles2': [parts[1]], 'ratio': [parts[2]]})
        df_generate = pd.concat([df_generate, new_row], ignore_index=True)

In [10]:
df_generate

,smiles1,smiles2,ratio
0,O(C(c1ccc(C(=O)*)cc1)=O)CCO*,C(O*)(=O)c1ccc(cc1)OC(=O)*,80
1,N(*)CC(=O)NCCCCCCNC(=O)CC*,C(CCNC(c1ccc(C(N*)=O)cc1)=O)CCC*,80
2,C(COC(c1ccc(cc1)C(=O)O*)=O)*,c1cc(C(O*)=O)ccc1C(OCCCC*)=O,84
3,c1cccc(Sc2cccc(C(*)=O)c2)cc1O*,c1(O*)ccc(C(c2ccc(cc2)*)=O)cc1,80
4,C(OC)(C(*)(C*)C)=O,c1cccc(COC(C(C)(*)C*)=O)c1,98
...,...,...,...
4912,c1c(ccc(c1)OCC(COC(=O)C(*)(C*)C)O)(O)C,c12ccc(cc2ccc1OCCOC(=O)OCCOC(=O)c1ccc(C(C)(C)c...,85
4913,C(CCCCO*)O*,C1(C(C)(C)C(C1OC(=O)c1ccc(C(O*)=O)cc1)C)O*)(C)C,70
4914,C(C(OCCOP(O)(O)=O)(C*)(C)C)=O,C(C*)(c1ccccc1)*,85
4915,c1c(ccc(c1)C(=O)O*)C(=O)OCC*,O=[N+](OCC1COC2C1OCC2*)c1ccc(C(=O)O*)cc1,92
